<a href="https://colab.research.google.com/github/Insisted/CH2-PS020_FitSync/blob/main/ML/model/Workout-Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    from google.colab import drive
    # import zipfile

    # drive.mount('/content/drive')

    # token = '/content/drive/MyDrive/BANGKIT/kaggle.json'
    # dataset_path = '/content/megaGymDataset.csv'

    # !pip install kaggle --quiet

    # !mkdir ~/.kaggle
    # !cp "{token}" ~/.kaggle/
    # !chmod 600 ~/.kaggle/kaggle.json

    # !kaggle datasets download -d niharika41298/gym-exercise-data

    # zip_ref = zipfile.ZipFile('gym-exercise-data.zip', 'r')
    # zip_ref.extractall('/content')
    # zip_ref.close()

    !gdown "1jKEs_oVjHQ7Ur6WgtZ7TdKd2enUU3L-b" # megaGymDataset.csv
    !gdown "1VmHL0uGVMhEWrVmz6GNNINRyQ8CR-JBz" # Gym-Visual-EXERCISES-list.xlsx

    dataset_path = './megaGymDataset.csv'
    xls_path = './Gym-Visual-EXERCISES-list.xlsx'
except ImportError:
    dataset_path = '../data/megaGymDataset.csv'
    xls_path = '../data/Gym-Visual-EXERCISES-list.xlsx'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(dataset_path) # https://www.kaggle.com/datasets/niharika41298/gym-exercise-data
df_xls = pd.read_excel(xls_path, sheet_name='Animated GIFs')

# megaGymDataset.csv

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2918 non-null   int64  
 1   Title       2918 non-null   object 
 2   Desc        1368 non-null   object 
 3   Type        2918 non-null   object 
 4   BodyPart    2918 non-null   object 
 5   Equipment   2886 non-null   object 
 6   Level       2918 non-null   object 
 7   Rating      1031 non-null   float64
 8   RatingDesc  862 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 205.3+ KB


In [5]:
df.isna().sum()

Unnamed: 0       0
Title            0
Desc          1550
Type             0
BodyPart         0
Equipment       32
Level            0
Rating        1887
RatingDesc    2056
dtype: int64

In [6]:
# Convert to categorical
df['Title'] = pd.Categorical(df.Title)
df['Type'] = pd.Categorical(df.Type)
df['BodyPart'] = pd.Categorical(df.BodyPart)
df['Level'] = pd.Categorical(df.Level)

df = df.loc[(df.Equipment == 'Body Only')] # (data.Level == 'Beginner'), (data.Desc.notna())

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078 entries, 185 to 2909
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed: 0  1078 non-null   int64   
 1   Title       1078 non-null   category
 2   Desc        404 non-null    object  
 3   Type        1078 non-null   category
 4   BodyPart    1078 non-null   category
 5   Equipment   1078 non-null   object  
 6   Level       1078 non-null   category
 7   Rating      269 non-null    float64 
 8   RatingDesc  196 non-null    object  
dtypes: category(4), float64(1), int64(1), object(3)
memory usage: 144.2+ KB


In [7]:
df.drop(['Unnamed: 0', 'Desc', 'RatingDesc'], axis=1, inplace=True)

In [8]:
df.head()

,Title,Type,BodyPart,Equipment,Level,Rating
185,Elbow plank,Strength,Abdominals,Body Only,Intermediate,9.3
186,Bottoms Up,Strength,Abdominals,Body Only,Intermediate,9.3
187,Spider crawl,Strength,Abdominals,Body Only,Intermediate,9.2
188,Cocoons,Strength,Abdominals,Body Only,Intermediate,9.1
189,Cross-Body Crunch,Strength,Abdominals,Body Only,Intermediate,9.1


# Gym-Visual-EXERCISES-list.xlsx

In [9]:
df_xls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5657 entries, 0 to 5656
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             5657 non-null   int64 
 1   Name           5657 non-null   object
 2   Exercise type  5643 non-null   object
 3   BodyPart       5640 non-null   object
 4   Equipment      5650 non-null   object
 5   Gender         5650 non-null   object
 6   Target         4 non-null      object
 7   Synergist      4 non-null      object
dtypes: int64(1), object(7)
memory usage: 353.7+ KB


In [10]:
df_xls.isna().sum()

ID                  0
Name                0
Exercise type      14
BodyPart           17
Equipment           7
Gender              7
Target           5653
Synergist        5653
dtype: int64

In [11]:
df_xls.drop(['ID', 'Target', 'Synergist'], axis=1, inplace=True)
df_xls.rename(columns = {'Exercise type': 'Type', 'Name': 'Title'}, inplace=True)

In [12]:
# Convert to categorical
df_xls['Title'] = pd.Categorical(df_xls.Title)
df_xls['Type'] = pd.Categorical(df_xls.Type)
df_xls['BodyPart'] = pd.Categorical(df_xls.BodyPart)
df_xls['Gender'] = pd.Categorical(df_xls.Gender)

df_xls = df_xls.loc[(df_xls.Equipment == 'Body weight')]

df_xls.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2212 entries, 0 to 5656
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Title      2212 non-null   category
 1   Type       2207 non-null   category
 2   BodyPart   2202 non-null   category
 3   Equipment  2212 non-null   object  
 4   Gender     2212 non-null   category
dtypes: category(4), object(1)
memory usage: 221.1+ KB


In [13]:
df_xls.head()

,Title,Type,BodyPart,Equipment,Gender
0,3/4 Sit-up,Strength,Waist,Body weight,Male
1,45 Side Bend,Strength,Waist,Body weight,Male
2,Air bike,Strength,Waist,Body weight,Male
3,Air Twisting Crunch,Strength,Waist,Body weight,Female
5,Alternate Heel Touchers,Strength,Waist,Body weight,Male
